In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

val hotspotBenchmarkLines = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("JmhBenchmark\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
hotspotBenchmarkLines.joinToString("\n")

baselineDouble: 931858.124 ± 2707.644 ns/op
baselineFloat: 928123.143 ± 12428.765 ns/op
inlineDouble: 930382.882 ± 6153.134 ns/op
inlineFloat: 900125.64 ± 3895.555 ns/op
longPackFloat: 1547488.939 ± 2048.011 ns/op
mutableRefDouble: 663923.418 ± 1923.981 ns/op
mutableRefFloatIn1Long: 1535125.654 ± 1731.307 ns/op
mutableRefFloatIn2Longs: 660791.922 ± 1326.528 ns/op
valueDouble: 656028.937 ± 884.5 ns/op
valueFloat: 667792.942 ± 18023.542 ns/op
valueInlineDouble: 653461.746 ± 6090.136 ns/op
valueInlineFloat: 674163.867 ± 16781.843 ns/op

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(platform: String, benchmarkLines: List<BenchmarkData>) {
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = listOf("Float", "Double").associateWith { type ->
        benchmarkLines.single { it.name == "baseline$type" }
    }
    val plot = letsPlot() +
            labs("Microbenchmark of 3D scene rendering ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 0.5),
                size = 5,
            ) {
                x = names
                y = scores
                label = benchmarkLines.map {
                    val isFloat = it.name.contains("Float")
                    val type = if (isFloat) "Float" else "Double"
                    val score = it.score / baseline[type]!!.score
                    val error = it.error / baseline[type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to scoresPlusErrors.max(), name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$platform.svg")
    plot.show()
}


In [4]:
makePlot("Hotspot", hotspotBenchmarkLines)

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

val artBenchmarkLines = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} \\d+:\\d+:\\d+.\\d+ \\d+ \\d+ I Benchmark: AndroidBenchmark.([^\\[]+)\\[Metric \\(timeNs\\) results: median ([^,]+), min [^,]+, max [^,]+, standardDeviation: ([^,]+), .*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = hotspotBenchmarkLines.mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         lines.sortedBy { indexes[it.name]!! }
     }
artBenchmarkLines.joinToString("\n")

baselineDouble: 1.3673541214285715E7 ± 464189.40463535936 ns/op
baselineFloat: 1.10071105625E7 ± 447734.83275383944 ns/op
inlineDouble: 2.9116641E7 ± 511091.9874000335 ns/op
inlineFloat: 2.3972317375E7 ± 789086.517194392 ns/op
longPackFloat: 4589821.404761905 ± 181221.69948283318 ns/op
mutableRefDouble: 4071994.395833333 ± 94852.11880345125 ns/op
mutableRefFloatIn1Long: 5817654.970588235 ± 90707.2457200662 ns/op
mutableRefFloatIn2Longs: 3926813.84 ± 364146.09072056296 ns/op
valueDouble: 1.6103076916666666E7 ± 527015.1355947905 ns/op
valueFloat: 1.0287985055555556E7 ± 310452.581539556 ns/op
valueInlineDouble: 851587.8884297521 ± 32680.665557614568 ns/op
valueInlineFloat: 807025.7384615385 ± 31382.075228020225 ns/op

In [6]:
makePlot("ART", artBenchmarkLines)